# 평가

## 분류
예측 값에 대한 결과 값이 맞냐 아니냐 대해 명확하게 구분하려 함.

## 회귀
예측 값과 결과의 차이가 가 작으면 정확하다 판단.

## 정확도
전체 데이터에서 예측해서 맞춘 결과의 퍼센트

- 오류
: 이진 분류로 데이터의 불균형이 심한 데이터를 학습하면 엄청난 정확도를 나타내지만 다른 데이터로 예측할 때 정확도가 매우 오차가 커진다.
따라서 정확도만 보고 모델을 검증하면 안된다.

In [10]:
# 학습은 안하고 예측만 하게 만들어봄

from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np
# 클래스 이름 붙여줘야 한다. 클래스 명은 보통 첫 단어부터 첫 글자를 대문자로 사용.
# 이 클래스는 상속 받을 때 다중 클래스 상속 가능하다.
class MyDummyClassifier(BaseEstimator):
    # 함수에서 fit 정의할 때는 파라미터로 self라고 정의를 하지만 
    # 대중적으로 self라고 적어줌 self 안에 들어간 주소 수정하기 빡세니까 수정 안하는게 좋음.
    # 나중에 어차피 활용 안됨.
    def fit(self,X,y=None):
        pass 
    
    # 여기선 fit에 아무것도 없어서 학습을 안한다.
    def predict(self,X):
        # 행은 데이터 건수 만큼, 열은 결과값만 받으면 되니까 1
        pred = np.zeros((X.shape[0],1))
        for i in range(X.shape[0]):
            # 들어오는 x값의 성별이 해당 인덱스로 가도록 설정
            if X['Sex'].iloc[i] == 1:
                pred[i] = 0
            else:
                pred[i] = 1
        return pred

In [11]:
# 데이터 수정할 내용 바꿔주고
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df

def drop_features(df):
    df.drop(columns=['PassengerId','Name','Ticket'],inplace=True)
    return df

def format_features(df):
    #레이블 인코더는 함수 안에 같이 넣어주는 것이 좋다.
    from sklearn.preprocessing import LabelEncoder
    
    df['Cabin'] = df['Cabin'].str[:1]
    features=['Sex','Cabin','Embarked']
    for feature in features:
        le = LabelEncoder()
        df[feature] = le.fit_transform(df[feature])
        # 데이터 몇 개 안되면 그냥 숫자값에 뭐가 있는지 보고 작업하면 좋다.
        print(le.classes_)
    return df

def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

df = pd.read_csv('titanic.csv')
y = df['Survived']
X = df.drop(columns=['Survived'])
X = transform_features(X)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=11)

['female' 'male']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'N' 'T']
['C' 'N' 'Q' 'S']


In [12]:
myclf = MyDummyClassifier()
myclf.fit(X_train,y_train)
pred = myclf.predict(X_test)
accuracy_score(y_test,pred)

0.8324022346368715

# 오차행렬
정확도가 높아도 실제로 정확하지 않을 수 있어서 오차행렬로 신뢰도를 확인해 볼 수 있다.
1이라고 예측을 할 때, 오차 행렬에서의 오차 데이터를 보고 정확도의 신뢰도를 확인한다.

In [13]:
from sklearn.metrics import confusion_matrix

In [14]:
confusion_matrix(y_test,pred)

array([[103,  15],
       [ 15,  46]], dtype=int64)

### 정확도
예측 결과와 실제 값이 동일한 건수 / 전체 데이터 수

## 정밀도와 재현율
혼동행렬에서 나온다.
이 수치는 동시에 올라갈 수가 없다. 어느 하나가 높아지면 나머지 하나가 낮아진다.

### 정밀도(Precision) = 양성 예측도
예측 값 기준으로
내가 1이라고 예측해서 실제로 1이라고 맞춘 비율
ex. 스팸 메일 거르다가 내 정상 메일까지 걸러버리면 안되기 때문에 정밀도가 더 중요.

### 재현율(Recall) = 민감도 = TPR
실제 값 기준으로
실제 1인 결과들에서 1이라고 예측에 성공한 비율
ex. 암환자 구분 모델에서 암환자 놓치면 안되기 때문에 재현율이 더 중요.

*** 농산물 판별 프로젝트는 재현율이 더 높아야 한다.